In [208]:
import pandas as pd
import matplotlib.pyplot as plt

In [209]:
df = pd.read_csv('train.csv')
df.head()

,gift_id,gift_type,gift_category,gift_cluster,instock_date,stock_update_date,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,uk_date1,uk_date2,is_discounted,volumes,price
0,GF_11156,61,534,3942,2014-02-21 05:07:06.000,2016-11-09 15:49:51.000,3377,5221,504,1912,10,554,2014-02-24 08:07:06.000,2014-02-24 07:07:06.000,0,NaN,175.54
1,GF_11157,61,534,3942,2014-02-21 06:07:06.000,2016-11-11 13:49:51.000,3377,5221,504,1912,10,554,2014-02-22 07:07:06.000,2014-02-24 06:07:06.000,1,NaN,95.80
2,GF_15689,584,262,0,2014-02-21 09:30:21.000,2016-03-24 14:46:18.000,5290,1579,3203,1912,9,1578,2016-01-26 00:04:45.000,2016-03-18 02:00:00.000,1,NaN,107.35
3,GF_11155,61,534,3942,2014-02-22 05:07:06.000,2016-11-10 16:49:51.000,3377,5221,504,1912,10,554,2016-11-07 13:49:51.000,2016-11-06 04:00:00.000,0,NaN,172.90
4,GF_11158,61,534,3942,2014-02-22 07:07:06.000,2016-11-10 13:49:51.000,3377,5221,504,1912,9,554,2016-11-07 15:49:51.000,2016-11-06 01:00:00.000,1,NaN,77.72


In [210]:

df2 = df.drop(['instock_date', 'stock_update_date', 'uk_date1', 'uk_date2'], axis = 'columns')
df2.head()

,gift_id,gift_type,gift_category,gift_cluster,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,is_discounted,volumes,price
0,GF_11156,61,534,3942,3377,5221,504,1912,10,554,0,NaN,175.54
1,GF_11157,61,534,3942,3377,5221,504,1912,10,554,1,NaN,95.80
2,GF_15689,584,262,0,5290,1579,3203,1912,9,1578,1,NaN,107.35
3,GF_11155,61,534,3942,3377,5221,504,1912,10,554,0,NaN,172.90
4,GF_11158,61,534,3942,3377,5221,504,1912,9,554,1,NaN,77.72


In [211]:
df.shape

(20279, 17)

In [212]:
df2['volumes'] = df2["volumes"].fillna(value=df["volumes"].mean())
df2.isnull().sum()

gift_id          0
gift_type        0
gift_category    0
gift_cluster     0
lsg_1            0
lsg_2            0
lsg_3            0
lsg_4            0
lsg_5            0
lsg_6            0
is_discounted    0
volumes          0
price            0
dtype: int64

In [213]:
X = df2[['gift_type', 'gift_category', 'gift_cluster', 'lsg_1', 'lsg_2', 'lsg_3', 'lsg_4', 'lsg_5', 'lsg_6', 'is_discounted', 'volumes']]
X.head()
y = df2[['price']]

In [214]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [215]:
X_train.shape

(16223, 11)

In [216]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [217]:
y_predict = model.predict(X_test)
y_predict

array([[343.32323612],
       [ 19.94343341],
       [286.35239728],
       ...,
       [ 62.67769451],
       [241.57835098],
       [105.37649462]])

In [218]:
model.score(X_test, y_test)

0.3624025630683113

In [219]:
from sklearn.metrics import accuracy_score
dft = pd.read_csv('test.csv')
dft = dft.drop(['instock_date', 'stock_update_date', 'uk_date1', 'uk_date2','gift_id'], axis = 'columns')
dft['volumes'] = dft["volumes"].fillna(value=dft["volumes"].mean())
dft.head()

,gift_type,gift_category,gift_cluster,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,is_discounted,volumes
0,842,663,24,203,1334,4358,1912,10,1899,0,15.457509
1,407,534,754,4785,5940,2582,1912,10,1488,0,15.457509
2,631,433,6012,1812,5027,3427,1912,10,1913,1,15.457509
3,992,433,6358,2002,6883,5537,1912,10,1734,0,15.457509
4,43,534,6123,1512,6883,610,1912,9,150,1,15.457509


In [220]:
y_predict2 = model.predict(dft)
y_predict2.shape

(13519, 1)

In [221]:
dft1 = pd.read_csv('test.csv')
dft1['value'] = y_predict2
submission = pd.DataFrame({'gift_id':dft1['gift_id'],'price':dft1['value']})
submission.head()

,gift_id,price
0,GF_2372,228.467855
1,GF_13040,108.847335
2,GF_5754,-20.106014
3,GF_1417,7.956031
4,GF_23204,-73.043201


In [222]:
filename = 'pricepred.csv'
submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: pricepred.csv
